In [1]:
import numpy as np
import pandas as pd

from plotnine import *

## Tagging

In [2]:
from estnltk import Text
from estnltk.taggers import NerTagger
from estnltk.web_taggers import NerWebTagger
from estnltk.converters import json_to_text

In [3]:
ner_tagger = NerTagger()
ner_web_tagger = NerWebTagger()

text = json_to_text(file='diff-ner/text.json')
text

Text(text='Berliini müür langes , kuid nüüd lahutab okastraat setosid , kuuleb välisminister Ilves setude tõde\n\nTOOMAS SILDAM , toomas.sildam@postimees.ee\n\nTäna nõuab Ain Sarv ( 51 ) välisminister Ilveselt aru , miks riik on setud unustanud , eile koputas ta erinevate firmade ustele , et küsida sponsorabi Põhjamaade rahvamuusika tudengite ja õppejõudude Viljandi kokkutulekule .\nToomas Huik\n\nTäna saabub kauaoodatud küsimise tund , kui Seto Kongressi Vanemate Kogu peavanem Ain Sarv küsib välisminister Toomas Hendrik Ilveselt : “ Härra minister , kas olete valmis oma kodanike õigused jalge alla tallama , et Eesti saaks Schengeni viisaruumi ? ”\n\nLäbitungiva pilgu ja kaheharulise habemega Ain Sarv ( 51 ) ei küsi seda retooriliselt ajalehe arvamuskülje repliigis , vaid välisministri kabinetis , näost näkku kohtumisel .\nIlves soostus Seto Kongressi Vanemate Kogu esinduse vastu võtma , sest suhted Toompea ja setude vahel on sama okkalised nagu sägadega traat , mille Vene piirivalve tõmbas seitse aastat tagasi piki Eesti NSV administratiivpiiri .\n\nNimekirjad , millega seitse ja pool tuhat setut kirikupühadel lihtsalt üle piiri Petseri poolele pääses , saadab valitsus paberihuntidele toiduks ning Eesti võimud nõustuvad euroametnikega , kes meilt kagupiiril tavalist viisarezhiimi nõuavad .\n\nHääletades Pihkvasse\n\nEsimest korda nägi 15-aastane Ain Setumaad läbi tiheda udu .\nSuvel 1963 sõitis ta koolivaheajal õe juurest Elvast pöidlaküüdiga Riiga , sest seal müüdi pöördkondensaatorit - taskuraadio ehitamiseks vajalikku vidinat .\nRiiast põrutas Ain “ häälega ” Pihkvasse , kuni hommikupoole ööd peatus Aini pöidlaviibutuse peale hele Pobeda .\n\n“ Riia-Pihkva kiviteel oli nii õudne udu , et juht ja mina keerasime aknad alla , pistsime pead välja ning vaatasime , kas ikka tee peal oleme .\nTagant hakkasid järsku paistma võimsad tuled justkui oleks rong tulnud .\nOli hoopis tugevate laternatega autobuss , mis meist nagu postist mööda põrutas .\nPobeda juht võttis end selle stopptulede kiiluvette , nii jõudsime lõpuks Pihkvasse . ”\n\nPäevavalges sõitis Ain uuesti läbi Setumaa , Elva poole .\nAga oskas üks Tallinna koolipoiss siis näha , et Petseri ümbrus ja sealsed inimesed on erilised ?\n\nAini silmad avanesid 1970. aasta tantsupeol .\nEnne sattus Sarvede pere noorim poeg Henn lugema kuulutust , et orkester vajab torupillimängijaid .\nEi Henn ega Ain polnud kordagi torupilli käes hoidnud , aga kaks lugu said selgeks ja tantsupeo rongkäigus plaksutasid tuhanded inimesed neile nagu elukutselistele rahvamuusikutele .\n\nAin üllatas siis tuttavaid , kui mõistis Värska tantsumemmede setukeelset juttu , sest Mehkamaal sündinud isa oli pojale õpetanud võro keelt ja Mulgimaalt pärit vanaema mulkide murdesõnu .\nVärska memmed kutsusid Aini koos torupilliansambli juhi Igor Tõnuristiga külla ning nii nad seisidki 70nda soojal suvel kahekesi setude lauluringis .\nVoolas veel veidi vett Pihkva järve ja Ain istus juba setude ühe esilauliku Veera Pähnapuu kõrval pingil , üle Põrste küla madalate majade kaikus : “ Laul läts läbi Setomaa , hõpõhelme helinäl . ”\n\nDetsembris 1970 loodud rahvakunstiansamblis Leegajus laulis Ain setude lugusid Igor Tõnuristi juhatuse all hiljem kümneid , kui mitte sadu kordi .\nSetumaa oli avastatud !\n\n“ Meeldis mulle see seto tüdruk , oi , kuidas meeldis , ” tunneb Ain tänini mõnusalt sooja kriipivat käppa südame lähedal , meenutades esimest kohtumist Piibarite ansambli Õiega .\n“ Tema kaudu ma selle rahva hulka pääsesingi . ”\n\nÕie tädi Anne ja tädimees Peeter elasid Petserimaal Pööni külas , aga siis enam teid nende talu juurde talviti lahti ei aetud , Anne-Peeter müüsid vana majapidamise ja ostsid uue Rissova külla , päris Petseri-Irboska tee äärde , kuni Peeter suri .\nKahe aasta eest kolis Anne venna juurde Eestisse , Oonurme .\n\n“ Elupäevi polnud tädi Annele aga palju antud .\nTa suri koduigatsusest , sest kontrolljoon oli juba vahel ja meie ei saanud enam talle Rissovasse appi käia .\nKui praegu tädi

## Postgres

In [4]:
from estnltk.storage.postgres import PostgresStorage, delete_schema

See [compose.yml](./diff-ner/compose.yml)

In [5]:
storage = PostgresStorage(host='localhost', port=5432, dbname='dev',
                          user='dev', password='dev', schema='ner_diff',
                          create_schema_if_missing=True)

INFO:storage.py:57: connecting to host: 'localhost', port: 5432, dbname: 'dev', user: 'dev'
INFO:storage.py:78: new schema 'ner_diff' created
INFO:storage.py:108: schema: 'ner_diff', temporary: False, role: 'dev'


In [6]:
collection = storage.add_collection('ner_diff_sampling')

with collection.insert() as collection_insert:
    for text_obj in [text]:
        ner_tagger.tag(text_obj)
        ner_web_tagger.tag(text_obj)
        
        collection_insert(text_obj)

INFO:storage.py:233: new empty collection 'ner_diff_sampling' created
INFO:collection_text_object_inserter.py:104: inserted 1 texts into the collection 'ner_diff_sampling'


See [run.sh](./diff-ner/run.sh)

## Sampling

In [7]:
storage.delete_collection(collection.name)
delete_schema(storage)
storage.close()

## Label studio